In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import norm
import json
from collections import Counter
import random
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import train_test_split
import math

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Import walking probability
walking_probability = pd.read_csv ('drive/MyDrive/Amenities/output/walking_probability.csv', index_col = False)
car_probability = pd.read_csv ('drive/MyDrive/Amenities/output/car_probability.csv', index_col = False)

In [ ]:
num_amenities=4
city = np.random.randint(2+num_amenities, size=(5,5))
city

array([[0, 1, 1, 3, 2],
       [1, 1, 2, 4, 0],
       [1, 1, 4, 2, 1],
       [4, 0, 4, 5, 1],
       [1, 2, 0, 3, 5]])

In [ ]:
# number 1 for houses, number 2 for restaurants
count_amenities=[0] * (num_amenities+1)
#import pdb
for k in range(len(count_amenities)):
  #pdb.set_trace()
  for i in range(len(city)):
    for j in range(len(city)):
      if city[i,j]==(k+1):
        count_amenities[k]=count_amenities[k]+1
count_amenities


[9, 4, 2, 4, 2]

In [ ]:
#Information (row,column) of every amenity
info=pd.DataFrame()
info['id']=""
info['type']=""
info['row']=""
info['column']=""
info['house_pop']=""

# number 1 for houses, number 2 for restaurants
pos=0

#import pdb
for k in range(len(count_amenities)):
  #pdb.set_trace()
  id=0
  for i in range(len(city)):
    for j in range(len(city)):
      if city[i,j]==(k+1):
        info.at[pos, 'id']=id
        info.at[pos, 'type']=k+1
        info.at[pos, 'row']=i
        info.at[pos, 'column']=j
        pos=pos+1
        id=id+1
        if info.at[pos-1, 'type']==1:
          info.at[pos-1, 'house_pop']=np.random.randint(50) #Asigno poblacion aleatoria a cada casa
      

info.head(10)

,id,type,row,column,house_pop
0,0,1,0,1,6
1,1,1,0,2,7
2,2,1,1,0,1
3,3,1,1,1,25
4,4,1,2,0,42
5,5,1,2,1,21
6,6,1,2,4,10
7,7,1,3,4,19
8,8,1,4,0,41
9,0,2,0,4,NaN


#Otra forma (con "demanda") --> Generalizando para n amenities
En vez de calcular el indice para cada casa, calcularlo para cada amenity.

Por ejemplo, para los restaurantes. Calcular el % de casas que esta en el grupo 1, en el grupo 2, en el grupo 3... y esto puede ser lo equivalente a la demanda del paper? Se puede tener en cuenta incluso la poblacion de cada casa, para diferenciar una casa familiar de una torre de 24 pisos.

In [ ]:
info['type'].nunique()

3

In [ ]:
l=0.1 #lado de cada celda en la mesa. Un cuadrado equivale a 0.05 millas
num_houses=len(info[info['type']==1])
num_amen=info['type'].nunique()-1 #Hay que quitar las casas


distance_df=pd.DataFrame()
distance_df['house_id']=""
distance_df['amen_id']=""
distance_df['amen_type']=""
distance_df['distance']=""

pos=0
list_houses=info[info['type']==1].reset_index(drop=True)
for i in range(num_houses):
  for k in range(2, num_amen+2):
    list_amen=info[info['type']==k].reset_index(drop=True)
    for j in range(len(list_amen)):
      x_dif=list_houses['row'][i]-list_amen['row'][j]
      y_dif=list_houses['column'][i]-list_amen['column'][j]
      d=math.sqrt(x_dif**2+y_dif**2)*l
      distance_df.at[pos, 'house_id']=i
      distance_df.at[pos, 'amen_id']=j
      distance_df.at[pos, 'amen_type']=k
      distance_df.at[pos, 'distance']=d
      pos=pos+1

distance_df.head(2)

,house_id,amen_id,amen_type,distance
0,0,0,2,0.3
1,0,1,2,0.141421


In [ ]:
distance_df['distance_group'] = pd.cut(distance_df['distance'], bins =[0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, np.inf]) # Age group
distance_df.head(3)

,house_id,amen_id,amen_type,distance,distance_group
0,0,0,2,0.3,"(0.25, 0.5]"
1,0,1,2,0.141421,"(0.0, 0.25]"
2,0,2,2,0.282843,"(0.25, 0.5]"


In [ ]:
house_info=info[info['type']==1].rename(columns = {'id': 'house_id'}).reset_index(drop=True)
house_info.head(7)

,house_id,type,row,column,house_pop
0,0,1,0,1,6
1,1,1,0,2,7
2,2,1,1,0,1
3,3,1,1,1,25
4,4,1,2,0,42
5,5,1,2,1,21
6,6,1,2,4,10


In [ ]:
house_info['house_pop'].sum()

172

In [ ]:
distance_df=distance_df.merge(house_info[['house_id', 'house_pop']], how='left', on='house_id')
distance_df.head(2)

,house_id,amen_id,amen_type,distance,distance_group,house_pop
0,0,0,2,0.3,"(0.25, 0.5]",6
1,0,1,2,0.141421,"(0.0, 0.25]",6


In [ ]:
count_houses_per_group = distance_df.groupby(['amen_type', 'distance_group'])['house_pop'].sum().to_frame().reset_index().rename(columns = {'house_pop': 'number_people'})
count_houses_per_group.head(10)

,amen_type,distance_group,number_people
0,2,"(0.0, 0.25]",315
1,2,"(0.25, 0.5]",332
2,2,"(0.5, 0.75]",41
3,2,"(0.75, 1.0]",0
4,2,"(1.0, 1.25]",0
5,2,"(1.25, 1.5]",0
6,2,"(1.5, inf]",0
7,3,"(0.0, 0.25]",77
8,3,"(0.25, 0.5]",267
9,3,"(0.5, 0.75]",0


In [ ]:
count_houses_per_group = distance_df.groupby(['amen_type', 'distance_group'])['house_pop'].sum().to_frame().reset_index().rename(columns = {'house_pop': 'number_people'})
total_people=count_houses_per_group.groupby(['amen_type'])['number_people'].sum().to_frame().reset_index().rename(columns = {'number_people': 'total_people'})
count_houses_per_group=count_houses_per_group.merge(total_people, how='left', on='amen_type')
count_houses_per_group['% demand']=count_houses_per_group['number_people']/count_houses_per_group['total_people']*100
count_houses_per_group.head(10)

,amen_type,distance_group,number_people,total_people,% demand
0,2,"(0.0, 0.25]",315,688,45.784884
1,2,"(0.25, 0.5]",332,688,48.255814
2,2,"(0.5, 0.75]",41,688,5.959302
3,2,"(0.75, 1.0]",0,688,0.0
4,2,"(1.0, 1.25]",0,688,0.0
5,2,"(1.25, 1.5]",0,688,0.0
6,2,"(1.5, inf]",0,688,0.0
7,3,"(0.0, 0.25]",77,344,22.383721
8,3,"(0.25, 0.5]",267,344,77.616279
9,3,"(0.5, 0.75]",0,344,0.0


In [ ]:
mean_distance_per_group = distance_df.groupby(['amen_type', 'distance_group'])['distance'].mean().to_frame().reset_index().rename(columns = {'distance': 'mean_distance'})
mean_distance_per_group.head(7)

,amen_type,distance_group,mean_distance
0,2,"(0.0, 0.25]",0.173076
1,2,"(0.25, 0.5]",0.341370
2,2,"(0.5, 0.75]",0.565685
3,2,"(0.75, 1.0]",NaN
4,2,"(1.0, 1.25]",NaN
5,2,"(1.25, 1.5]",NaN
6,2,"(1.5, inf]",NaN


In [ ]:
score_info=pd.DataFrame()
score_info=count_houses_per_group.merge(mean_distance_per_group, how='left', on=['amen_type', 'distance_group'])
score_info

,amen_type,distance_group,number_people,total_people,% demand,mean_distance
0,2,"(0.0, 0.25]",315,688,45.784884,0.173076
1,2,"(0.25, 0.5]",332,688,48.255814,0.341370
2,2,"(0.5, 0.75]",41,688,5.959302,0.565685
3,2,"(0.75, 1.0]",0,688,0.0,NaN
4,2,"(1.0, 1.25]",0,688,0.0,NaN
5,2,"(1.25, 1.5]",0,688,0.0,NaN
6,2,"(1.5, inf]",0,688,0.0,NaN
7,3,"(0.0, 0.25]",77,344,22.383721,0.185374
8,3,"(0.25, 0.5]",267,344,77.616279,0.363633
9,3,"(0.5, 0.75]",0,344,0.0,NaN


In [ ]:
score_info['probability']=""

n=score_info['amen_type'].nunique() #diferentes tipos de amenities (en este caso 1 porque solo tenemos restaurantes)

for j in range(len(score_info)):
  amen_type=score_info.at[j, 'amen_type']
  if amen_type==2: sub_amen='Full-Service Restaurants'
  if amen_type==3: sub_amen= 'Clothing Store'
  if amen_type==4: sub_amen= 'Elementary School'
  if amen_type==5: sub_amen= 'Museum'
  score_info.at[j, 'probability']=walking_probability.iloc[(walking_probability['distance']-score_info.at[j, 'mean_distance']).abs().argsort()[:1]][sub_amen].reset_index(drop=True)[0]
score_info

,amen_type,distance_group,number_people,total_people,% demand,mean_distance,probability
0,2,"(0.0, 0.25]",315,688,45.784884,0.173076,0.835543
1,2,"(0.25, 0.5]",332,688,48.255814,0.341370,0.735175
2,2,"(0.5, 0.75]",41,688,5.959302,0.565685,0.60268
3,2,"(0.75, 1.0]",0,688,0.0,NaN,0.079131
4,2,"(1.0, 1.25]",0,688,0.0,NaN,0.079131
5,2,"(1.25, 1.5]",0,688,0.0,NaN,0.079131
6,2,"(1.5, inf]",0,688,0.0,NaN,0.079131
7,3,"(0.0, 0.25]",77,344,22.383721,0.185374,0.911203
8,3,"(0.25, 0.5]",267,344,77.616279,0.363633,0.85198
9,3,"(0.5, 0.75]",0,344,0.0,NaN,0.064007


In [ ]:
score_info['multiply']=score_info['% demand']*score_info['probability']
score_info.head(2)

,amen_type,distance_group,number_people,total_people,% demand,mean_distance,probability,multiply
0,2,"(0.0, 0.25]",425,820,51.829268,0.172229,0.835543,43.305561
1,2,"(0.25, 0.5]",395,820,48.170732,0.348527,0.735175,35.413902


In [ ]:
walkability = score_info.groupby(['amen_type'])['multiply'].sum().to_frame().reset_index().rename(columns = {'multiply': 'walkability_score'})
walkability.head(7)

,amen_type,walkability_score
0,2,78.719463
1,3,88.334333


# % of people travelling by bus, car and walking

In [ ]:
#Import walking probability
frequency = pd.read_csv ('drive/MyDrive/Amenities/output/BUA_analysis_month2.csv', index_col = False)
frequency.head(2)

,top_amenity,sub_amenity,count_POIs,frequency,count_visits,visits_pPOI,NPOI_p1000,mean_sqm,median_sqm,min_sqm,max_sqm,total_sqm,Nsqm_p1000
0,Cultural,Art Gallery,24,0.064648,35646.0,1485.250000,0.043527,440.2,505.0,150.0,695.0,2201.0,3.991752
1,Cultural,Library,13,0.029903,16488.0,1268.307692,0.023577,1641.0,1729.0,133.0,3164.0,13128.0,23.809049


In [ ]:
travel_mode=distance_df.copy()

In [ ]:
cond1 = {
    2: 'Full-Service Restaurants',
    3:'Clothing Store'
}

travel_mode['sub_amenity'] = travel_mode['amen_type'].map(cond1)
travel_mode['sub_amenity'] = travel_mode['amen_type'].map(cond1)
travel_mode.head(2)

,house_id,amen_id,amen_type,distance,distance_group,house_pop,sub_amenity
0,0,0,2,0.3,"(0.25, 0.5]",6,Full-Service Restaurants
1,0,1,2,0.141421,"(0.0, 0.25]",6,Full-Service Restaurants


In [ ]:
travel_mode=travel_mode.merge(frequency[['sub_amenity', 'frequency']], how='left', on='sub_amenity')
travel_mode.head(2)

,house_id,amen_id,amen_type,distance,distance_group,house_pop,sub_amenity,frequency
0,0,0,2,0.3,"(0.25, 0.5]",6,Full-Service Restaurants,2.388192
1,0,1,2,0.141421,"(0.0, 0.25]",6,Full-Service Restaurants,2.388192


In [ ]:
## Seleccionar el amen_id más cercano para cada amen_type
num_houses=len(info[info['type']==1])

for j in range(travel_mode['sub_amenity'].nunique()):
  aux_amen=travel_mode[travel_mode['sub_amenity']==travel_mode['sub_amenity'].unique()[j]]
  for i in range(num_houses):
    aux=aux_amen[aux_amen['house_id']==i]
    min_distance=aux['distance'].min()
    pos=aux.index[aux['distance']==min_distance][0]
    travel_mode.at[pos, 'min?']=1


travel_mode.head(10)

,house_id,amen_id,amen_type,distance,distance_group,house_pop,sub_amenity,frequency,min?
0,0,0,2,0.3,"(0.25, 0.5]",6,Full-Service Restaurants,2.388192,NaN
1,0,1,2,0.141421,"(0.0, 0.25]",6,Full-Service Restaurants,2.388192,1.0
2,0,2,2,0.282843,"(0.25, 0.5]",6,Full-Service Restaurants,2.388192,NaN
3,0,3,2,0.4,"(0.25, 0.5]",6,Full-Service Restaurants,2.388192,NaN
4,0,0,3,0.2,"(0.0, 0.25]",6,Clothing Store,0.366810,1.0
5,0,1,3,0.447214,"(0.25, 0.5]",6,Clothing Store,0.366810,NaN
6,0,0,4,0.223607,"(0.0, 0.25]",6,NaN,NaN,NaN
7,0,1,4,0.223607,"(0.0, 0.25]",6,NaN,NaN,NaN
8,0,2,4,0.316228,"(0.25, 0.5]",6,NaN,NaN,NaN
9,0,3,4,0.316228,"(0.25, 0.5]",6,NaN,NaN,NaN


In [ ]:
travel_def=travel_mode[travel_mode['min?']==1].reset_index(drop=True)
travel_def.head(10)

,house_id,amen_id,amen_type,distance,distance_group,house_pop,sub_amenity,frequency,min?
0,0,1,2,0.141421,"(0.0, 0.25]",6,Full-Service Restaurants,2.388192,1.0
1,0,0,3,0.2,"(0.0, 0.25]",6,Clothing Store,0.366810,1.0
2,1,1,2,0.1,"(0.0, 0.25]",7,Full-Service Restaurants,2.388192,1.0
3,1,0,3,0.1,"(0.0, 0.25]",7,Clothing Store,0.366810,1.0
4,2,1,2,0.2,"(0.0, 0.25]",1,Full-Service Restaurants,2.388192,1.0
5,2,0,3,0.316228,"(0.25, 0.5]",1,Clothing Store,0.366810,1.0
6,3,1,2,0.1,"(0.0, 0.25]",25,Full-Service Restaurants,2.388192,1.0
7,3,0,3,0.223607,"(0.0, 0.25]",25,Clothing Store,0.366810,1.0
8,4,1,2,0.223607,"(0.0, 0.25]",42,Full-Service Restaurants,2.388192,1.0
9,4,0,3,0.360555,"(0.25, 0.5]",42,Clothing Store,0.366810,1.0


In [ ]:
travel_def['walk_probability']=""

n=score_info['amen_type'].nunique() #diferentes tipos de amenities (en este caso 1 porque solo tenemos restaurantes)

for j in range(len(travel_def)):
  sub_amen=travel_def.at[j, 'sub_amenity']
  travel_def.at[j, 'walk_probability']=walking_probability.iloc[(walking_probability['distance']-travel_mode.at[j, 'distance']).abs().argsort()[:1]][sub_amen].reset_index(drop=True)[0]
  travel_def.at[j, 'car_probability']=car_probability.iloc[(car_probability['distance']-travel_mode.at[j, 'distance']).abs().argsort()[:1]][sub_amen].reset_index(drop=True)[0]
travel_def.head(2)

,house_id,amen_id,amen_type,distance,distance_group,house_pop,sub_amenity,frequency,min?,walk_probability,car_probability
0,0,1,2,0.141421,"(0.0, 0.25]",6,Full-Service Restaurants,2.388192,1.0,0.763531,0.141839
1,0,0,3,0.2,"(0.0, 0.25]",6,Clothing Store,0.366810,1.0,0.925611,0.053503


In [ ]:
travel_def['walk_trips']=travel_def['house_pop']*travel_def['frequency']*travel_def['walk_probability']
travel_def['car_trips']=travel_def['house_pop']*travel_def['frequency']*travel_def['car_probability']
travel_def.head(5)

,house_id,amen_id,amen_type,distance,distance_group,house_pop,sub_amenity,frequency,min?,walk_probability,car_probability,walk_trips,car_trips
0,0,1,2,0.141421,"(0.0, 0.25]",6,Full-Service Restaurants,2.388192,1.0,0.763531,0.141839,10.940749,2.032434
1,0,0,3,0.2,"(0.0, 0.25]",6,Clothing Store,0.366810,1.0,0.925611,0.053503,2.037137,0.117753
2,1,1,2,0.1,"(0.0, 0.25]",7,Full-Service Restaurants,2.388192,1.0,0.763531,0.141839,12.764207,2.371173
3,1,0,3,0.1,"(0.0, 0.25]",7,Clothing Store,0.366810,1.0,0.825992,0.084545,2.120871,0.217084
4,2,1,2,0.2,"(0.0, 0.25]",1,Full-Service Restaurants,2.388192,1.0,0.813715,0.127162,1.943307,0.303687


In [ ]:
trips_by_mode=pd.DataFrame()
trips_by_mode['walk']=""
trips_by_mode['car']=""
trips_by_mode.at[0,'walk']=travel_def['walk_trips'].sum() #.to_frame().reset_index().rename(columns = {'multiply': 'walkability_score'})
trips_by_mode.at[0, 'car']=travel_def['car_trips'].sum()
trips_by_mode=trips_by_mode.transpose().rename(columns = {0: 'n_trips'})
trips_by_mode.head(4)

,n_trips
walk,381.568545
car,58.795423


In [ ]:
trips_by_mode['% trips']=trips_by_mode['n_trips']/trips_by_mode['n_trips'].sum()*100
trips_by_mode.head(2)

,n_trips,% trips
walk,381.568545,86.648448
car,58.795423,13.351552
